### Importing

In [2]:
import pandas as pd
import spacy
import contextualSpellCheck
import neuspell
from neuspell import BertChecker
from neuspell import SclstmChecker
import spellchecker
import enchant
from sentence_transformers import SentenceTransformer
import numpy as np
from numpy.linalg import norm
import nltk


c:\Users\gaura\AppData\Local\Programs\Python\Python38\lib\site-packages\cupy\_environment.py:205: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\Users\gaura\AppData\Local\Programs\Python\Python38\lib\site-packages\cupy\_environment.py:205: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\Users\gaura\AppData\Local\Programs\Python\Python38\lib\site-packages\cupy\_environment.py:205: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\Users\gaura\AppData\Local\Programs\Python\Python38\lib\site-packages\cupy\_environment.py:205: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\Users\gaura\AppData\Local\Programs\Python\Python38\lib\site-packages\cupy\_environment.py:205: UserWarning: CUDA path cou

data folder is set to `c:\Users\gaura\AppData\Local\Programs\Python\Python38\lib\site-packages\neuspell\../data` script


### Some initial Pre-processing (Not Important)

In [14]:
check_address = pd.read_csv('Dataset/list_of_real_usa_addresses.csv')
real_address = pd.read_csv('uscities.csv')

In [15]:
city = []
state = []
city_state_check = []
for add in check_address['city']:
    city.append(add)

for address in check_address['state']:
    state.append(address)

for something in range(0,len(check_address['city'])):
    city_state_check.append(city[something] + ',' + state[something])


In [16]:
city = []
state = []
city_state_real = []
for add in real_address['city']:
    city.append(add)

for address in real_address['state_id']:
    state.append(address)

for something in range(0, len(real_address['city'])):
    city_state_real.append(city[something] + ',' + state[something])

In [ ]:
city_state_check
print(city_state_real)

In [81]:
usa_addresses = pd.read_excel('usa_address.xlsx')

In [82]:
usa_addresses['city_state']= city_state_real

In [84]:
usa_addresses['city_state']

0           New York,NY
1        Los Angeles,CA
2            Chicago,IL
3              Miami,FL
4             Dallas,TX
              ...      
28333          Gross,NE
28334         Lotsee,OK
28335      The Ranch,MN
28336       Shamrock,OK
28337         Monowi,NE
Name: city_state, Length: 28338, dtype: object

In [88]:
usa_addresses

,Unnamed: 0,city,state_name,state_id,city_state
0,0,New York,New York,NY,"New York,NY"
1,1,Los Angeles,California,CA,"Los Angeles,CA"
2,2,Chicago,Illinois,IL,"Chicago,IL"
3,3,Miami,Florida,FL,"Miami,FL"
4,4,Dallas,Texas,TX,"Dallas,TX"
...,...,...,...,...,...
28333,28333,Gross,Nebraska,NE,"Gross,NE"
28334,28334,Lotsee,Oklahoma,OK,"Lotsee,OK"
28335,28335,The Ranch,Minnesota,MN,"The Ranch,MN"
28336,28336,Shamrock,Oklahoma,OK,"Shamrock,OK"


In [90]:
usa_addresses.to_excel('usa_addresses.xlsx')

In [18]:
usa_address.columns
usa_addresses = real_address[['city', 'state_name', 'state_id']]


In [19]:
usa_addresses.to_excel('usa_address.xlsx')

### Address Correction (Leveshtein Distance)

In [114]:
sample = pd.read_csv('Dataset/list_of_real_usa_addresses.csv')
predefined_dataset = pd.read_excel('usa_address.xlsx')

#### Comparing with Cities and States together

In [112]:
city_state_sample=[]
for city in range(0,len(sample)):
    sample_city = sample['city'][city]
    sample_state = sample['state'][city]

    string_output = str(sample_city)+','+str(sample_state)
    city_state_sample.append(string_output)

city_state_predefined = predefined_dataset['city_state']
city_state_predefined = list(city_state_predefined)


In [113]:
not_present_city_state = []

for city in city_state_sample:
    if city not in city_state_predefined:
        not_present_city_state.append(city)
print(f"Length - {len(not_present_city_state)}")

Length - 85


#### Comparing with City and State different

In [115]:
sample_cities = []
sample_city = sample['city']
sample_city = list(sample_city)

predefined_city = predefined_dataset['city']
predefined_city = list(predefined_city)

for city in sample_city:
    if city not in predefined_city:
        sample_cities.append(city)


In [116]:
sample_states = []

sample_state = sample['state']
sample_state = list(sample_state)

predefined_state = predefined_dataset['state_id']
predefined_state = list(predefined_state)

for state in sample_state:
    if state not in predefined_state:
        sample_states.append(state)



In [117]:
not_present_city_state = []

for city in range(0,len(sample_city)):

    if sample_city[city] in sample_cities:
        string1 = sample_city[city]+','+sample_state[city]
        not_present_city_state.append(string1)


In [118]:
for state in range(0,len(sample_state)):
    
    if sample_state[state] in sample_states:
        string1 = sample_city[state]+','+sample_state[state]
        not_present_city_state.append(string1)

not_present_city_state = set(not_present_city_state)


In [123]:
cities_corrected = {}
minimum = 999
for iterator_city_state in not_present_city_state:
    minimum = 999
    for correct_city_state in city_state_predefined:
        temp = enchant.utils.levenshtein( iterator_city_state, correct_city_state)
        if temp < 5:
            if temp < minimum:
                minimum = temp
                cities_corrected[iterator_city_state] = correct_city_state

        else:
            continue


In [124]:
cities_corrected

{'Albany,N': 'Albany,NY',
 'Bellinghm,MN': 'Bellingham,MN',
 'Ware,MA': 'Wade,MS',
 'Wareham,MA': 'Waltham,MA',
 'Northhampton,MA': 'Northampton,MA',
 'Abington,MA': 'Abingdon,VA',
 'Norh Adms,MA': 'North Adams,MA',
 'East Falmth,MA': 'East Falmouth,MA',
 'Salem,A': 'Salem,MA'}

### Bigram Method:

check_cities = check_address['city'] 
check_state = check_address['state']
official_cities = list(usa_addresses['city'])
official_state_id = list(usa_addresses['state_id'])

In [46]:
def cosine_function(embeddings1, embeddings2):  # Cosine Function

    A = embeddings1
    B = embeddings2
    cosine = np.dot(A, B)/(norm(A)*norm(B))
    return cosine


model_path = 'local/path/to/model'  # Loading the model
model = SentenceTransformer(model_path)


In [8]:
city_state = []
for city in range(0,len(check_cities)):
    if check_cities[city] not in official_cities:
        string1 = check_cities[city] + ',' + check_state[city]
        city_state.append(string1)

    elif check_state[city] not in official_state_id:
        string2 = check_state[city] + ',' +check_state[city]
        city_state.append(string2)


In [20]:
city_state

['Abington,MA',
 'Bellinghm,MN',
 'Chelmsford,MA',
 'East Falmth,MA',
 'Leicester,MA',
 'Methuen,MA',
 'Norh Adms,MA',
 'North Attleboro,MA',
 'North Dartmouth,MA',
 'North Oxford,MA',
 'North Reading,MA',
 'Northborough,MA',
 'Northhampton,MA',
 'A,A',
 'Seekonk,MA',
 'Sturbridge,MA',
 'Tewksbury,MA',
 'Walpole,MA',
 'Ware,MA',
 'Wareham,MA',
 'N,N',
 'Camillus,NY',
 'Canandaigua,NY',
 'Catskill,NY',
 'Cheektowaga,NY',
 'Cobleskill,NY',
 'Cortlandville,NY',
 'East Greenbush,NY',
 'Fishkill,NY',
 'Greece,NY',
 'Halfmoon,NY',
 'Herkimer,NY',
 'Horseheads,NY',
 'Lowville,NY',
 'Macedon,NY',
 'Mohegan Lake,NY',
 'East Windsor,CT',
 'Old Saybrook,CT',
 'Southington,CT']

In [21]:
len(city_state)

39

In [10]:
model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')

In [42]:
city = 'Abington, MA'
correct_city = 'New York, NY'


In [43]:
encode1 = model.encode([city])[0]
encode2 = model.encode([correct_city])[0]


In [35]:
def jaccard_similarity(encode1, encode2):
    
    np.asarray(encode1, e)
    intersection = np.logical_and( encode1, encode2)
    # print(intersection)
    union = np.logical_or(encode1, encode2 )

    return intersection.sum()/float(union.sum())
    

In [44]:
similarity_percentage = cosine_function( encode1, encode2)
# similarity_percentage = jaccard_similarity(encode1, encode2)
# print(encode1)
# print(similarity_percentage)

In [19]:
for city in city_state:
    print(city)
    minimum=0
    encode1 = model.encode([city])[0]
    for correct_city in city_state_real:
        encode2 = model.encode([correct_city])[0]
        similarity_percentage = cosine_function( encode1, encode2)
        # print(similarity_percentage)
        # print(correct_city)
        if similarity_percentage>=0.80:
            # print(city)
            print('In')
            if similarity_percentage > minimum:
                minimum = similarity_percentage
                cities_corrected[city] = correct_city



Abington,MA
Bellinghm,MN
In


NameError: name 'cities_corrected' is not defined

In [ ]:
for city in city_state:
    if cities_corrected[city]==None:
        cities_corrected[city]= 'Nothing Found'

In [ ]:
cities_corrected

{'Abington': 'Abingdon',
 'Chelmsford': 'Elmsford',
 'Leicester': 'Worcester',
 'Methuen': 'Meriden',
 'North Attleboro': 'North Attleborough',
 'North Dartmouth': 'North Falmouth',
 'North Oxford': 'Port Orford',
 'North Reading': 'Port Reading',
 'Northborough': 'Northboro',
 'Northhampton': 'Northampton',
 'Seekonk': 'Sedona',
 'Sturbridge': 'Stockbridge',
 'Tewksbury': 'Lewisburg',
 'Walpole': 'Waldorf',
 'Ware': 'Ward',
 'Wareham': 'Waltham',
 'Camillus': 'Camilla',
 'Canandaigua': 'Catasauqua',
 'Catskill': 'Cresskill',
 'Cheektowaga': 'Chetopa',
 'Cobleskill': 'Noblesville',
 'Cortlandville': 'Collinsville',
 'East Greenbush': 'East Greenville',
 'Fishkill': 'Highfill',
 'Greece': 'Greene',
 'Halfmoon': 'Half Moon',
 'Herkimer': 'Herriman',
 'Horseheads': 'Morehead',
 'Lowville': 'Lionville',
 'Macedon': 'Macon',
 'Mohegan Lake': 'Moses Lake',
 'East Windsor': 'East Bangor',
 'Old Saybrook': 'Saybrook',
 'Southington': 'Worthington'}

In [31]:
print(len(not_present_city))
print(len(not_present_state))

0

In [ ]:
cities_corrected

### Weighted Bigram

#### Contextual Spell Check

In [6]:
nlp = spacy.load('en_core_web_sm')
contextualSpellCheck.add_to_pipe(nlp)

In [8]:
doc = nlp('Rishil wo the Qlondon elections.')
print('Corrected:', doc._.outcome_spellCheck)

Corrected: Rishil won the Qlondon elections.


#### Neural Spell Checker

In [ ]:
checker = BertChecker()
checker.from_pretrained()

In [16]:
checker.correct("Rishil wo the Qlondon elections.")

'Rishil to the Qlondon elections .'

In [35]:
from spellchecker import SpellChecker

spell = SpellChecker()

# find those words that may be misspelled
misspelled = spell.unknown(["Abingto", "Avenue", "study", "wrte"])

for word in misspelled:
    # Get the one `most likely` answer
    print(spell.correction(word))

    # Get a list of `likely` options
    print(spell.candidates(word))


bingo
{'goingto', 'bingo', 'aingt', 'thingto'}
write
{'write', 'wre', 'wrote', 'wite'}


In [ ]:
import difflib

a = 'Abington'
b = 'Abingdon'

seq = difflib.SequenceMatcher(None, a, b)
d = seq.ratio()*100
print(d)
### OUTPUT: 87.323943


87.5


In [55]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("oliverguhr/spelling-correction-english-base")
model = AutoModelForSeq2SeqLM.from_pretrained("oliverguhr/spelling-correction-english-base")


Downloading: 100%|██████████| 353/353 [00:00<00:00, 88.2kB/s]
Downloading: 100%|██████████| 798k/798k [00:01<00:00, 607kB/s]  
Downloading: 100%|██████████| 456k/456k [00:01<00:00, 452kB/s]  
Downloading: 100%|██████████| 2.11M/2.11M [00:01<00:00, 1.11MB/s]
Downloading: 100%|██████████| 239/239 [00:00<00:00, 238kB/s]
Downloading: 100%|██████████| 1.74k/1.74k [00:00<00:00, 1.72MB/s]
Downloading: 100%|██████████| 558M/558M [00:36<00:00, 15.3MB/s]   


In [4]:
from transformers import pipeline

fix_spelling = pipeline("text2text-generation",model="oliverguhr/spelling-correction-english-base")


[{'generated_text': 'Abingt.'}]


In [14]:
print(fix_spelling("Bellinghem",max_length=2048))

[{'generated_text': 'Bellinghem.'}]


In [1]:
import transformers

print(transformers.__version__)

4.25.1


In [1]:
import enchant

# determining the values of the parameters
string1 = "abc"
string2 = "aef"

# the Levenshtein distance between
# string1 and string2
print(enchant.utils.levenshtein(string1, string2))


2


### Abbreviation (Address)

In [3]:
abbreviation = pd.read_excel('Dataset/Abrreviations.xlsx')
sample = pd.read_csv('Dataset/list_of_real_usa_addresses.csv')

In [4]:
proper_abbreviations = abbreviation['Abbreviations']
proper_abbreviations = list(proper_abbreviations)

correct_abbreviations = abbreviation['Corrected_Abbreviations']
correct_abbreviations = list(correct_abbreviations)


In [ ]:
mapping_dict = {}
for something in range(0,len(proper_abbreviations)):
    mapping_dict[proper_abbreviations[something]] = correct_abbreviations[something]
    
mapping_dict

In [10]:
corrected_df = pd.DataFrame()
corrected_df['1'] = mapping_dict.keys()
corrected_df['2'] = mapping_dict.values()

In [11]:
corrected_df

,1,2
0,Ave,Avenue
1,Blvd,Boulevard
2,Cyn,Canyon
3,Dr,Drive
4,Ln,Lane
5,Rd,Road
6,St,Street
7,E,East
8,N,North
9,NE,North-East


In [182]:
Address = sample['address']
Address = list(Address)

In [183]:
tokens = []
abbrev = []

for part in Address:
    nltk_tokens = nltk.word_tokenize(part)
    tokens.append(nltk_tokens)

for part in tokens:
    for some in range(0,len(part)):
        if part[some] in mapping_dict.keys():
            part[some]= mapping_dict[part[some]]

for tok in range(0, len(tokens)):
    tokens[tok] = ' '.join(tokens[tok])


In [ ]:
import enchant
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spellchecker import SpellChecker
from fuzzywuzzy import fuzz


def checkAutoCorrect(tokenlist):
    '''This function checks whether words are english or not and corrects the spelling mistakes
    Input : list of token
    Output : corrected string
    '''
    temp = ''
    spell = SpellChecker(language='en')
    for token in tokenlist:
        # check for abbreviation
        if len(token) <= 3:
            temp = temp+" "+token
        # check if word is english
        elif dict.check(token):
            temp = temp+" "+token
        else:
            # correct the word if spelling mistakes
            correctword = str(spell.correction(token)) if spell.correction(token) else token
            temp = temp+" "+correctword

    original = ' '.join(tokenlist)
    print(f'original string:{original} , replaced string:{temp}, ratio: {fuzz.WRatio(original, temp)}')
    return temp.upper()

if __name__ == "__main__"  :
    dict = enchant.Dict("en_US")
    dataset1 = pd.read_csv("test.csv")
    dataset1 = pd.DataFrame(dataset1["Provider Name"], columns=["Provider Name"])
    # tokenization of provider name
    dataset1["tokenized Name"] = [list(word_tokenize(x)) for x in dataset1["Provider Name"]]
    # correction  string
    dataset1["Filtered Name"] = [checkAutoCorrect(x) for x in dataset1["tokenized Name"]]
    print(dataset1)




In [184]:
for tok in tokens:
    print(tok)

777 Brockton Avenue
30 Memorial Drive
250 Hartford Avenue
700 Oak Street
66-4 Parkhurst Road
591 Memorial Drive
55 Brooksby Village Way
137 Teaticket Highway
42 Fairhaven Commons Way
374 William South Canning Boulevard
121 Worcester Road
677 Timpany Boulevard
337 Russell Street
295 Plymouth Street
1775 Washington Street
280 Washington Street
20 Soojian Drive
11 Jungle Road
301 Massachusetts Avenue
780 Lynnway
70 Pleasant Valley Street
830 Curran Memorial Highway
1470 South Washington Street
506 State Road
742 Main Street
72 Main Street
200 Otis Street
180 North King Street
555 East Main Street
555 Hubbard Ave-Suite 12
300 Colony Place
301 Falls Boulevard
36 Paramount Drive
450 Highland Avenue
1180 Fall River Avenue
1105 Boston Road
100 Charlton Road
262 Swansea Mall Drive
333 Main Street
550 Providence Highway
352 Palmer Road
3005 Cranberry Highway Rt 6 28
250 Rt 59
141 Washington Avenue Extension
13858 Rt 31 West
2055 Niagara Falls Boulevard
101 Sanford Farm Shpg Center
297 Grant Aven

### Spell Corrector - (Address)

In [175]:
!pip install textblob
!pip install jamspell
!pip install autocorrect
!pip install Symspell

You should consider upgrading via the 'C:\Users\gaura\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.



  Using cached jamspell-0.0.12.tar.gz (174 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for jamspell
Failed to build jamspell
    Running setup.py install for jamspell: started
    Running setup.py install for jamspell: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\gaura\AppData\Local\Programs\Python\Python38\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\gaura\\AppData\\Local\\Temp\\pip-install-2as2bd9y\\jamspell_65f5512d22764ac8a97b2a21f389daae\\setup.py'"'"'; __file__='"'"'C:\\Users\\gaura\\AppData\\Local\\Temp\\pip-install-2as2bd9y\\jamspell_65f5512d22764ac8a97b2a21f389daae\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\gaura\AppData\Local\Temp\pip-wheel-moui73_o'
       cwd: C:\Users\gaura\AppData\Local\Temp\pip-install-2as2bd9y\jamspell_65f5512d22764ac8a97b2a21f389daae\
  Complete output (45 lines):
  C:\Users\gaura\AppData\Local\Programs\Python\Python38\lib\site-packages\setuptoo

In [173]:
from textblob import TextBlob

txt = 'Hllo'
b = TextBlob(txt)
print(b.correct())

All
